### Import and Settings

In [19]:
import pandas as pd

pd.set_option('display.max_rows', 500)

### Read in files and reformat

In [20]:
ustat = pd.read_csv("csv files/ustat_2020.csv")
futbin = pd.read_csv("csv files/futbin_2020.csv")

In [21]:
futbin = futbin.rename(columns={"name": "player_name", "club": "team_title"})

In [22]:
ustat.team_title = ustat.team_title.str.split(',')
ustat = ustat.explode('team_title').reset_index(drop=True)

## Dataset matching

* Utilizes the club and players ids to match the understat and futbin datasets
* Remove duplicates and missing data from result then rename columns to match database

In [23]:
unpack_ex = pd.read_csv("data repair csvs/fix_club_ids.csv")
unpack_ex = unpack_ex.rename(columns={"db_id": "db_club_id"})
ustat = ustat.merge(unpack_ex[['us_team', 'db_club_id']], how='left', left_on='team_title', right_on='us_team')
futbin = futbin.merge(unpack_ex[['futbin_id', 'db_club_id']], how='left', left_on='club_id', right_on='futbin_id')

In [24]:
unpack_nam = pd.read_csv("data repair csvs/fix_name_ids.csv")
unpack_nam = unpack_nam.drop(['futbin_name', 'score'], axis=1)
unpack_nam = unpack_nam.rename(columns={"id": "player_id", "ustat_name": "player_name"})
ustat = ustat.merge(unpack_nam, how='left', on='player_name')

In [25]:
comb = pd.merge(futbin, ustat, how='left', on=['player_id', 'db_club_id'])

In [26]:
comb[comb.shots.isna()]

,player_name_x,pos,card_id,year,link,player_id,dob,team_title_x,club_id,league,...,key_passes,yellow_cards,red_cards,position,team_title_y,npg,npxG,xGChain,xGBuildup,us_team
0,Jürgen Locadia,ST,30443,21,/21/player/30443/jurgen-locadia,204366,07-11-1993,Brighton,1808,Premier League,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Winston Reid,CB,30389,21,/21/player/30389/winston-reid,176285,03-07-1988,West Ham,19,Premier League,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yangel Herrera,CM,30382,21,/21/player/30382/yangel-herrera,234060,07-01-1998,Manchester City,10,Premier League,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Juan Foyth,CB,9772,21,/21/player/9772/juan-foyth,237221,12-01-1998,Spurs,18,Premier League,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Ryan Sessegnon,LM,9491,21,/21/player/9491/ryan-sessegnon,235883,18-05-2000,Spurs,18,Premier League,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Gedson Fernandes,CM,9199,21,/21/player/9199/gedson-fernandes,234568,09-01-1999,Spurs,18,Premier League,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Marko Grujić,CM,8674,21,/21/player/8674/marko-grujic,232099,13-04-1996,Liverpool,9,Premier League,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Fabián Balbuena,CB,8005,21,/21/player/8005/fabian-balbuena,228229,23-08-1991,West Ham,19,Premier League,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Lovre Kalinic,GK,6789,21,/21/player/6789/lovre-kalinic,220932,03-04-1990,Aston Villa,2,Premier League,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Wesley Hoedt,CB,6448,21,/21/player/6448/wesley-hoedt,216774,06-03-1994,Southampton,17,Premier League,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
cleaned = comb[comb.shots.notna()]
cleaned = cleaned.drop_duplicates()
cleaned = cleaned.drop('us_team', axis=1)
cleaned = cleaned.rename(columns={"player_name_x": "player_name_futbin", "team_title_x": "club_name_futbin",
                                 "player_name_y": "player_name_ustat", "team_title_y": "club_name_ustat",
                                 "id": "player_id_ustat", "club_id": "club_id_futbin", 'pos': 'position',
                                 'db_club_id': 'club_id', 'year':'season'})

In [11]:
# To have correct null value in the database
cleaned = cleaned.fillna('NULL')

In [12]:
# Export complete file into a csv
cleaned.to_csv("csv files/cleaned_2021.csv",index=False)